In [ ]:
import pandas as pd


# 1.Extract Data

In [ ]:
data=pd.read_csv("data/uber_data.csv",sep=',')

In [ ]:
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)
data['trip_id'] = data.index

In [69]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
0,1,2016-01-03,2016-01-03 00:07:00,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,0
1,1,2016-01-03,2016-01-03 00:11:00,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,1
2,2,2016-01-03,2016-01-03 00:31:00,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,2
3,2,2016-01-03,2016-01-03 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,3
4,2,2016-01-03,2016-01-03 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,4


## Data-Warehouse-schema 
<img src="Data Model (2).png" alt="Data modeling" width="700" height="400"/>


# 2.Transform Data

### Create datetime_dim Table

In [ ]:
datetime_dim = data[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
###################
datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
###################
datetime_dim['datetime_id'] = datetime_dim.index
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]


In [68]:
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-01-03,0,3,1,2016,6,2016-01-03 00:07:00,0,3,1,2016,6
1,1,2016-01-03,0,3,1,2016,6,2016-01-03 00:11:00,0,3,1,2016,6
2,2,2016-01-03,0,3,1,2016,6,2016-01-03 00:31:00,0,3,1,2016,6
3,3,2016-01-03,0,3,1,2016,6,2016-01-03 00:00:00,0,3,1,2016,6
4,4,2016-01-03,0,3,1,2016,6,2016-01-03 00:00:00,0,3,1,2016,6


In [ ]:
datetime_dim.to_csv('datetime_dim.csv', index=False)


### Create passenger_count_dim Table

In [ ]:
passenger_count_dim = data[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

In [70]:
passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1
1,1,1
2,2,2
3,3,3
4,4,5


### Create trip_distance_dim Table

In [ ]:
trip_distance_dim = data[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [71]:
trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,4,30.43


In [ ]:
trip_distance_dim.to_csv('trip_distance_dim.csv', index=False)
passenger_count_dim.to_csv('passenger_count_dim.csv', index=False)

### Create rate_code_dim Table

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = data[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

In [72]:
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


In [ ]:
rate_code_dim.to_csv('rate_code_dim.csv', index=False)

### Create pickup_location_dim Table

In [ ]:
pickup_location_dim = data[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 

In [76]:
pickup_location_dim.head()

,pickup_location_id,pickup_latitude,pickup_longitude
0,0,40.765152,-73.976746
1,1,40.767925,-73.983482
2,2,40.644810,-73.782021
3,3,40.769814,-73.863419
4,4,40.792183,-73.971741


### Create dropoff_location_dim Table

In [ ]:
dropoff_location_dim = data[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

In [75]:
dropoff_location_dim.head()

,dropoff_location_id,dropoff_latitude,dropoff_longitude
0,0,40.746128,-74.004265
1,1,40.733166,-74.005943
2,2,40.675770,-73.974541
3,3,40.757767,-73.969650
4,4,40.695053,-74.177170


### Create payment_type_dim Table

In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = data[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [77]:
payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


In [ ]:
pickup_location_dim.to_csv('pickup_location_dim.csv', index=False)
dropoff_location_dim.to_csv('dropoff_location_dim.csv', index=False)
payment_type_dim.to_csv('payment_type_dim.csv', index=False)


### Create trip_fact_table Table

In [ ]:

trip_fact_table = data.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [73]:
trip_fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [ ]:
trip_fact_table.to_csv('trip_fact_table.csv', index=False)

In [ ]:
import pyodbc

# 3.Load Data

### Connect to SQL server database

In [ ]:
pyodbc.drivers()

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=AHMEDYOUSEFF;'
                      'Database=Taxi_TripDWH;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

### Load data from dataframe to sql server database

In [50]:
for row in datetime_dim.itertuples():
    cursor.execute('''
                INSERT INTO datetime_dim (datetime_id, tpep_pickup_datetime, 
                   pick_hour,pick_day,pick_month,pick_year,pick_weekday,
                   tpep_dropoff_datetime,drop_hour,drop_day,drop_month,
                   drop_year,drop_weekday)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                row.datetime_id, 
                row.tpep_pickup_datetime,
                row.pick_hour,
                row.pick_day,
                row.pick_month, 
                row.pick_year,
                row.pick_weekday,
                row.tpep_dropoff_datetime,
                row.drop_hour,
                row.drop_day,
                row.drop_month,
                row.drop_year,
                row.drop_weekday,
                )
conn.commit()

In [53]:
for row in dropoff_location_dim.itertuples():
    cursor.execute('''
                INSERT INTO dropoff_location_dim (dropoff_location_id, dropoff_latitude, 
                   dropoff_longitude)
                VALUES (?,?,?)
                ''',
                row.dropoff_location_id, 
                row.dropoff_latitude,
                row.dropoff_longitude,
                )
conn.commit()

In [55]:
for row in passenger_count_dim.itertuples():
    cursor.execute('''
                INSERT INTO passenger_count_dim (passenger_count_id, passenger_count)
                VALUES (?,?)
                ''',
                row.passenger_count_id, 
                row.passenger_count,
                )
conn.commit()

In [57]:
for row in payment_type_dim.itertuples():
    cursor.execute('''
                INSERT INTO payment_type_dim (payment_type_id, payment_type, 
                   payment_type_name)
                VALUES (?,?,?)
                ''',
                row.payment_type_id, 
                row.payment_type,
                row.payment_type_name,
                )
conn.commit()

In [59]:
for row in pickup_location_dim.itertuples():
    cursor.execute('''
                INSERT INTO pickup_location_dim (pickup_location_id, pickup_latitude, 
                   pickup_longitude)
                VALUES (?,?,?)
                ''',
                row.pickup_location_id, 
                row.pickup_latitude,
                row.pickup_longitude,
                )
conn.commit()

In [61]:
for row in rate_code_dim.itertuples():
    cursor.execute('''
                INSERT INTO rate_code_dim (rate_code_id, RatecodeID, 
                   rate_code_name)
                VALUES (?,?,?)
                ''',
                row.rate_code_id, 
                row.RatecodeID,
                row.rate_code_name,
                )
conn.commit()

In [64]:
for row in trip_distance_dim.itertuples():
    cursor.execute('''
                INSERT INTO trip_distance_dim (trip_distance_id, trip_distance)
                VALUES (?,?)
                ''',
                row.trip_distance_id, 
                row.trip_distance,
                )
conn.commit()

In [67]:
for row in trip_fact_table.itertuples():
    cursor.execute('''
                INSERT INTO trip_fact_table (trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,
                   dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                row.trip_id, 
                row.VendorID,
                row.datetime_id, 
                row.passenger_count_id,
                row.trip_distance_id, 
                row.rate_code_id,
                row.store_and_fwd_flag, 
                row.pickup_location_id,
                row.dropoff_location_id, 
                row.payment_type_id,
                row.fare_amount, 
                row.extra,
                row.mta_tax, 
                row.tip_amount,
                row.tolls_amount, 
                row.improvement_surcharge,
                row.total_amount,
                )
conn.commit()